In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

In [16]:
# !pip install beautifulsoup4

In [17]:
# !pip install lxml

In [18]:
# !pip install requests

# 1. Scrape webpage and create dataframe

In [111]:
from bs4 import BeautifulSoup
import requests

# Get html file
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

# create bs4 object and fetch the table
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_="wikitable sortable")

# clean the data: replace 'Not assigned' and merge duplicated postcode
maps = {}
for row in table.find_all('tr'):
    temp =[]
    for item in row.find_all('td'):
        temp.append(item.text.strip('\n'))        
    if len(temp) == 3:
        if temp[1] != 'Not assigned':
            if temp[2] != 'Not assigned':
                if (temp[0], temp[1]) not in maps.keys():
                    maps[(temp[0], temp[1])] = temp[2]
                else:
                    maps[(temp[0], temp[1])] = maps[(temp[0], temp[1])] + ', ' + temp[2]
            else:
                if (temp[0], temp[1]) not in maps.keys():
                    maps[(temp[0], temp[1])] = temp[1]
                    
# build the dataframe Series                    
df_series = {'Postcode':[], 'Borough':[], 'Neighbourhood':[]}
for k, v in maps.items():
    df_series['Postcode'].append(k[0])
    df_series['Borough'].append(k[1])
    df_series['Neighbourhood'].append(v)
    
df = pd.DataFrame(df_series)   
df = df.drop_duplicates(keep='first')
df.head()

Postcode           Borough                     Neighbourhood
0      M3A        North York                         Parkwoods
1      M4A        North York                  Victoria Village
2      M5A  Downtown Toronto                      Harbourfront
3      M6A        North York  Lawrence Heights, Lawrence Manor
4      M7A      Queen's Park                      Queen's Park

In [81]:
df.shape

(103, 3)

# 2. Derive latitude and the longitude coordinates

In [14]:
# !pip install geocoder

In [3]:
import geocoder # import geocoder
# geocoder not work, so I decide to use Geospatial_Coordinates.csv file to look up longitude and latitude

In [112]:
df_Geo = pd.read_csv('Geospatial_Coordinates.csv')
df2 = df.merge(df_Geo, how='left', left_on='Postcode', right_on='Postal Code').drop('Postal Code', axis=1)
df2.head()

Postcode           Borough                     Neighbourhood   Latitude  \
0      M3A        North York                         Parkwoods  43.753259   
1      M4A        North York                  Victoria Village  43.725882   
2      M5A  Downtown Toronto                      Harbourfront  43.654260   
3      M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4      M7A      Queen's Park                      Queen's Park  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

# 3. Explore and cluster the neighborhoods in Toronto

In [15]:
# !conda install -c conda-forge geopy --yes 

### (1) Use geopy library to get the latitude and longitude values of Toronto, Ontario

In [113]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


### (2) Create a map of Toronto, Ontario with neighborhoods superimposed on top

In [114]:
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm 
import matplotlib.colors as colors

In [115]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = f'{neighborhood},{borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### (3) Let's focus on neighborhoods in Downtown Toronto

In [116]:
df_downtown = df2[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
df_downtown.head()

Postcode           Borough             Neighbourhood   Latitude  Longitude
0      M5A  Downtown Toronto              Harbourfront  43.654260 -79.360636
1      M9A  Downtown Toronto              Queen's Park  43.667856 -79.532242
2      M5B  Downtown Toronto  Ryerson, Garden District  43.657162 -79.378937
3      M5C  Downtown Toronto            St. James Town  43.651494 -79.375418
4      M5E  Downtown Toronto               Berczy Park  43.644771 -79.373306

In [117]:
# creat the map for Downtown Toronto
address = "Downtown Toronto, Toronto"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [118]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### (4) Define Foursquare Credentials and Version

In [119]:
CLIENT_ID = '3EIW2ITW1J0JRZZANMKZ1AVEIW3QYUPWCRXJMS23ITAWLVRA' # your Foursquare ID
CLIENT_SECRET = 'PEZWGF3OTY12TZKYW4DX5JX5YYLQH0SRLINLY2LPR3SPXDH2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3EIW2ITW1J0JRZZANMKZ1AVEIW3QYUPWCRXJMS23ITAWLVRA
CLIENT_SECRET:PEZWGF3OTY12TZKYW4DX5JX5YYLQH0SRLINLY2LPR3SPXDH2


### (5) Find venues for each neighborhood

In [120]:
# Create a function to explore the venues of the neighorhood
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
# find venues for each neighborhood and show the process for each neighborhood
downtown_venues = getNearbyVenues(names=df_downtown['Neighbourhood'],
                                 latitudes=df_downtown['Latitude'],
                                 longitudes=df_downtown['Longitude'])
downtown_venues = downtown_venues[downtown_venues['Venue Category'] != 'Neighborhood']

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [122]:
print (downtown_venues.shape)
downtown_venues.head()

(1272, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Harbourfront               43.65426              -79.360636   
1  Harbourfront               43.65426              -79.360636   
2  Harbourfront               43.65426              -79.360636   
3  Harbourfront               43.65426              -79.360636   
4  Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653191       -79.357947   
3      Morning Glory Cafe       43.653947       -79.361149   
4          Impact Kitchen       43.656369       -79.356980   

         Venue Category  
0                Bakery  
1           Coffee Shop  
2  Gym / Fitness Center  
3        Breakfast Spot  
4            Restaurant

In [123]:
# show the number of venues for each neighborhood
downtown_venues.groupby(by=['Neighborhood'])['Venue'].count().to_frame()

Venue
Neighborhood                                             
Adelaide, King, Richmond                               99
Berczy Park                                            55
CN Tower, Bathurst Quay, Island airport, Harbou...     15
Cabbagetown, St. James Town                            44
Central Bay Street                                     85
Chinatown, Grange Park, Kensington Market              89
Christie                                               17
Church and Wellesley                                   85
Commerce Court, Victoria Hotel                        100
Design Exchange, Toronto Dominion Centre              100
First Canadian Place, Underground city                100
Harbord, University of Toronto                         35
Harbourfront                                           50
Harbourfront East, Toronto Islands, Union Station      99
Rosedale                                                4
Ryerson, Garden District                              100
St. James Town                                        100
Stn A PO Boxes 25 The Esplanade                        95

In [124]:
# find the number of unique venues in total
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 205 uniques categories.


### (6) Calculate the frequence of each venues in every Neighborhood

In [125]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

print ('The shape of the downtown_onehot is', downtown_onehot.shape)
downtown_onehot.head()

The shape of the downtown_onehot is (1272, 206)


Neighborhood  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0  Harbourfront                  0        0                   0             0   
1  Harbourfront                  0        0                   0             0   
2  Harbourfront                  0        0                   0             0   
3  Harbourfront                  0        0                   0             0   
4  Harbourfront                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop     ...       Thrift / Vintage Store  Toy / Game Store  Trail  \
0             0     ...                            0                 0      0   
1             0     ...                            0                 0      0   
2             0     ...                            0                 0      0   
3             0     ...                            0                 0      0   
4             0     ...                            0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  Yoga Studio  
0                      0         0            0            0  
1                      0         0            0            0  
2                      0         0            0            0  
3                      0         0            0            0  
4                      0         0            0            0  

[5 rows x 206 columns]

In [126]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped.head()

Neighborhood  Afghan Restaurant  \
0                           Adelaide, King, Richmond                0.0   
1                                        Berczy Park                0.0   
2  CN Tower, Bathurst Quay, Island airport, Harbo...                0.0   
3                        Cabbagetown, St. James Town                0.0   
4                                 Central Bay Street                0.0   

    Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0  0.000000            0.000000      0.000000        0.000000   
1  0.000000            0.000000      0.000000        0.000000   
2  0.066667            0.066667      0.066667        0.133333   
3  0.000000            0.000000      0.000000        0.000000   
4  0.000000            0.000000      0.000000        0.000000   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0         0.000000          0.000000             0.020202           0.0   
1         0.000000          0.000000             0.000000           0.0   
2         0.133333          0.133333             0.000000           0.0   
3         0.000000          0.000000             0.022727           0.0   
4         0.000000          0.000000             0.011765           0.0   

      ...       Thrift / Vintage Store  Toy / Game Store  Trail  \
0     ...                          0.0               0.0    0.0   
1     ...                          0.0               0.0    0.0   
2     ...                          0.0               0.0    0.0   
3     ...                          0.0               0.0    0.0   
4     ...                          0.0               0.0    0.0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0            0.0                       0.020202               0.0   
1            0.0                       0.018182               0.0   
2            0.0                       0.000000               0.0   
3            0.0                       0.000000               0.0   
4            0.0                       0.011765               0.0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  Yoga Studio  
0                    0.0  0.010101          0.0     0.000000  
1                    0.0  0.000000          0.0     0.000000  
2                    0.0  0.000000          0.0     0.000000  
3                    0.0  0.000000          0.0     0.000000  
4                    0.0  0.011765          0.0     0.011765  

[5 rows x 206 columns]

### (7)  Print each neighborhood along with the top 5 most common venues

In [127]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.08
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4      Salad Place  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1        Beer Bar  0.04
2            Café  0.04
3     Cheese Shop  0.04
4  Farmers Market  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4          Boutique  0.07


----Cabbagetown, St. James Town----
                venue  freq
0         Coffee Shop  0.07
1                Café  0.05
2  Italian Restaurant  0.05
3          Restaurant  0.05
4              Bakery  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.14
1   Italian Restaurant  0.05
2                 Café  0.04
3       Sandwich Place  0.0

### (8) Put that into a pandas dataframe

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [153]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']


# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2  CN Tower, Bathurst Quay, Island airport, Harbo...        Airport Lounge   
3                        Cabbagetown, St. James Town           Coffee Shop   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café            Steakhouse       Thai Restaurant   
1        Farmers Market    Seafood Restaurant                Bakery   
2       Airport Service      Airport Terminal       Harbor / Marina   
3           Pizza Place    Italian Restaurant                Bakery   
4    Italian Restaurant        Sandwich Place   Japanese Restaurant   

  5th Most Common Venue 6th Most Common Venue  
0           Salad Place          Burger Joint  
1              Beer Bar          Cocktail Bar  
2                   Bar      Sculpture Garden  
3            Restaurant                   Pub  
4        Ice Cream Shop          Burger Joint

### (9) Cluster Neighborhoods ( k-means )

In [154]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 4, 0, 0, 0], dtype=int32)

In [155]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
downtown_merged = downtown_merged.merge(neighborhoods_venues_sorted, how='inner', left_on='Neighbourhood', right_on='Neighborhood').drop('Neighborhood', axis=1)

downtown_merged.head() # check the last columns!

Postcode           Borough             Neighbourhood   Latitude  Longitude  \
0      M5A  Downtown Toronto              Harbourfront  43.654260 -79.360636   
1      M5B  Downtown Toronto  Ryerson, Garden District  43.657162 -79.378937   
2      M5C  Downtown Toronto            St. James Town  43.651494 -79.375418   
3      M5E  Downtown Toronto               Berczy Park  43.644771 -79.373306   
4      M5G  Downtown Toronto        Central Bay Street  43.657952 -79.387383   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0               0           Coffee Shop                  Park   
1               0           Coffee Shop        Clothing Store   
2               0                  Café           Coffee Shop   
3               0           Coffee Shop        Farmers Market   
4               0           Coffee Shop    Italian Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                Bakery                   Pub                  Café   
1        Cosmetics Shop  Fast Food Restaurant                  Café   
2            Restaurant                 Hotel        Clothing Store   
3    Seafood Restaurant                Bakery              Beer Bar   
4        Sandwich Place   Japanese Restaurant        Ice Cream Shop   

       6th Most Common Venue  
0             Breakfast Spot  
1  Middle Eastern Restaurant  
2             Cosmetics Shop  
3               Cocktail Bar  
4               Burger Joint

### (10) Visualize the resulting clusters

In [156]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters